In [1]:
import os
import sys

sys.dont_write_bytecode = True
os.environ["NUMBA_DISABLE_JIT"] = "1"


In [2]:
import numpy as np
import pandas as pd
from my_stuff import MufexKeys
from nb_quantfreedom.exchanges.mufex_exchange.mufex import Mufex
from nb_quantfreedom.nb_enums import DynamicOrderSettingsArrays
from nb_quantfreedom.utils import pretty_qf
from requests import get
np.set_printoptions(formatter={"float_kind": "{:.2f}".format})

pd.options.display.float_format = "{:.2f}".format

mufex_main = Mufex(
    api_key=MufexKeys.api_key,
    secret_key=MufexKeys.secret_key,
    use_test_net=False,
)

%load_ext autoreload
%autoreload 2


In [3]:
pd.DataFrame(get('https://api.mufex.finance/public/v1/market/kline?category=linear&symbol=BTCUSDT&interval=5').json()['data']['list'])

,0,1,2,3,4,5,6
0,1697768700000,28791,28804.4,28738.4,28763.1,7.341,211177.9197
1,1697769000000,28763.1,28926,28753.4,28888.2,30.297,874802.6115
2,1697769300000,28888.2,28920,28856,28893,13.88,401030.425
3,1697769600000,28893,28898.5,28850.7,28898.5,10.557,304842.3454
4,1697769900000,28898.5,29098.5,28898.5,28978.6,78.618,2280811.9963
...,...,...,...,...,...,...,...
195,1697827200000,29491.6,29494.9,29474.4,29489.3,1.561,46027.5359
196,1697827500000,29489.3,29489.5,29474.1,29479.5,0.971,28627.2393
197,1697827800000,29479.5,29491.3,29472.3,29472.3,1.149,33876.8514
198,1697828100000,29472.3,29486.1,29465.5,29469,1.628,47985.67


In [4]:
candles = mufex_main.get_candles_df(
    symbol="BTCUSDT",
    timeframe="5m",
    limit=1000
)
pd.DataFrame(candles)

It took 00 mins and 00 seconds to download 1000 candles


,0,1,2,3,4
0,1697528400000.00,28236.10,28331.80,28234.70,28325.80
1,1697528700000.00,28325.80,28374.30,28311.60,28373.80
2,1697529000000.00,28373.80,28452.20,28373.80,28452.20
3,1697529300000.00,28452.20,28452.60,28389.90,28405.80
4,1697529600000.00,28405.80,28466.20,28377.40,28466.10
...,...,...,...,...,...
995,1697826900000.00,29508.80,29529.00,29491.10,29491.60
996,1697827200000.00,29491.60,29494.90,29474.40,29489.30
997,1697827500000.00,29489.30,29489.50,29474.10,29479.50
998,1697827800000.00,29479.50,29491.30,29472.30,29472.30


In [5]:
d_arrays = DynamicOrderSettingsArrays(
    entry_size_asset=np.array([0]),
    max_equity_risk_pct=np.array([3]),
    max_trades=np.array([5]),
    num_candles=np.array([61]),
    risk_account_pct_size=np.array([.2,5]),
    risk_reward=np.array([64.5]),
    sl_based_on_add_pct=np.array([2]),
    sl_based_on_lookback=np.array([6]),
    sl_bcb_type=np.array([2]),
    sl_to_be_cb_type=np.array([1.948,6]),
    sl_to_be_when_pct=np.array([1000]),
    sl_to_be_ze_type=np.array([54.61]),
    static_leverage=np.array([153]),
    trail_sl_bcb_type=np.array([0.64654]),
    trail_sl_by_pct=np.array([1.20]),
    trail_sl_when_pct=np.array([156.6]),
)

In [6]:
1697767520 / 31556926

53.80015531297313